
# Atividade DISTRIBUTED DATA PROCESSING & STORAGE - RM353182 (Victor Hugo Eloi Hishikawa Ascencio)



## Importa bibliotecas

In [1]:
import pyspark
from delta import *
from delta.tables import *
from pyspark.sql.functions import *

builder = pyspark.sql.SparkSession.builder.appName("delta").master("spark://spark-master:7077") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key","datalake") \
    .config("spark.hadoop.fs.s3a.secret.key","datalake") \
    .config("spark.hadoop.fs.s3a.endpoint","http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8709e0be-8349-452e-b0e1-de020eed45ae;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.1.0/delta-spark_2.12-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.1.0!delta-spark_2.12.jar (4316ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.1.0/delta-storage-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.1.0!delta-storage.jar (373ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (560ms)
:: resolution report :: resolve 4280ms :: artifacts dl


## Extrai dados do arquivo Dados_1_.csv

In [3]:
# Local do arquivo
file_location = "s3a://bronze/Dados.csv"


# Configurando CSV
infer_schema = "false"
# A primeira coluna é o título: True (Verdadeiro)
first_row_is_header = "True"
# Delimitador da do arquivo: (",")
delimiter = ","

# Cria o de partidas utilizando o spark
df_partidas = spark.read.format('csv') \
 .option("inferSchema", infer_schema) \
 .option("header", first_row_is_header) \
 .option("sep", delimiter) \
 .load(file_location)
 
# Apresentando o df_partidas
display(df_partidas)

Py4JJavaError: An error occurred while calling o81.load.
: java.nio.file.AccessDeniedException: s3a://bronze/Dados.csv: getFileStatus on s3a://bronze/Dados.csv: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 184C6CEC690840A1; S3 Extended Request ID: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8; Proxy: null), S3 Extended Request ID: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8:403 Forbidden
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:255)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:175)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3796)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3688)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$exists$34(S3AFileSystem.java:4703)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.lambda


## Questões

In [ ]:
# 1 - Quantos registros existem na base?

display(df_partidas.count())

40839

In [ ]:
# 2 - Quantas equipes únicas mandantes existem na base?

display(df_partidas.select(countDistinct('home_team')))

count(DISTINCT home_team)
309


In [ ]:
# 3 - Quantas vezes as equipes mandantes saíram vitoriosas?

df_mandantes_vitoriosos = df_partidas.where(df_partidas.home_score > df_partidas.away_score).count()
display(df_mandantes_vitoriosos)

19856

In [ ]:
# 4 - Quantas vezes as equipes visitantes saíram vitoriosas?

df_visitantes_vitoriosos = df_partidas.where(df_partidas.home_score < df_partidas.away_score).count()
display(df_visitantes_vitoriosos)

11552

In [ ]:
#5 - Quantas partidas resultaram em empate?

df_empates = df_partidas.where(df_partidas.home_score == df_partidas.away_score).count()
display(df_empates)

9431

In [ ]:
# 6 - Quantas partidas foram realizadas em cada país?
df_qtd_partidas_paises = df.groupBy('country').count()
display(df_qtd_partidas_paises.orderBy('count', ascending=False))

country,count
United States,1144
France,801
England,687
Malaysia,644
Sweden,637
Germany,581
Brazil,529
Spain,517
Thailand,483
Italy,480


In [ ]:
# 7 - Qual país teve mais partidas?

display(df_qtd_partidas_paises.orderBy('count', ascending=False).take(1))

country,count
United States,1144


In [ ]:
# 8 - Qual a partida com maior número de gols?

df_partida_resutlado = df_partidas.withColumn('resultado',df_partidas.home_score + df_partidas.away_score).drop('neutral')

display(df_partida_resutlado.orderBy('resultado', ascending=False).take(1))


date,home_team,away_team,home_score,away_score,tournament,city,country,resultado
2001-04-11,Australia,American Samoa,31,0,FIFA World Cup qualification,Coffs Harbour,Australia,31.0


In [ ]:
# 9 - Qual a maior goleada?

df_partida_goleada = df_partidas.withColumn('resultado',df_partidas.home_score - df_partidas.away_score).drop('neutral')

display(df_partida_goleada.orderBy('resultado', ascending=False).take(1))

date,home_team,away_team,home_score,away_score,tournament,city,country,resultado
2001-04-11,Australia,American Samoa,31,0,FIFA World Cup qualification,Coffs Harbour,Australia,31.0


In [ ]:
# 10 - Quantos jogos ocorreram no Brasil?
df_qtd_partidas_paises.where(df_qtd_partidas_paises.country == 'Brazil').show()

+-------+-----+
|country|count|
+-------+-----+
| Brazil|  529|
+-------+-----+



In [4]:
spark.stop()